In [5]:
import re

import numpy as np
from tensorflow.keras.layers import Dense, SimpleRNN, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [6]:
with open('train_data_true.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace(r'\ufeff', '')
    
max_words_count = 1000
tokenizer = Tokenizer(num_words=max_words_count, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', char_level=False, lower=True, split=' ')

tokenizer.fit_on_texts([text])

In [7]:
inp_words = 3
data = tokenizer.texts_to_sequences([text])
res = to_categorical(data[0], num_classes=max_words_count)

n = res.shape[0] - inp_words

In [8]:
X = np.array([res[i:i + inp_words, :] for i in range(n)])
Y = res[inp_words:]

In [9]:
model = Sequential()
model.add(Input((inp_words, max_words_count)))
model.add(SimpleRNN(
    128, activation='tanh'
))

model.add(Dense(
    max_words_count, activation='softmax'
))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [10]:
history = model.fit(X, Y, epochs=32, batch_size=50)

Epoch 1/32
20/20 [==============================] - 3s 11ms/step - loss: 6.8996 - accuracy: 0.0041
Epoch 2/32
20/20 [==============================] - 0s 11ms/step - loss: 6.6935 - accuracy: 0.0872
Epoch 3/32
20/20 [==============================] - 0s 10ms/step - loss: 6.0566 - accuracy: 0.0365
Epoch 4/32
20/20 [==============================] - 0s 11ms/step - loss: 5.7771 - accuracy: 0.0558
Epoch 5/32
20/20 [==============================] - 0s 11ms/step - loss: 5.6649 - accuracy: 0.0568
Epoch 6/32
20/20 [==============================] - 0s 11ms/step - loss: 5.5444 - accuracy: 0.0477
Epoch 7/32
20/20 [==============================] - 0s 11ms/step - loss: 5.4096 - accuracy: 0.0619
Epoch 8/32
20/20 [==============================] - 0s 11ms/step - loss: 5.2537 - accuracy: 0.0629
Epoch 9/32
20/20 [==============================] - 0s 11ms/step - loss: 5.0745 - accuracy: 0.1166
Epoch 10/32
20/20 [==============================] - 0s 12ms/step - loss: 4.8537 - accuracy: 0.1511
Epoch 11/

In [11]:
def build_phrase(texts, str_len=20):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        x = to_categorical(data[i: i + inp_words], num_classes=max_words_count)  # преобразуем в One-Hot-encoding
        inp = x.reshape(1, inp_words, max_words_count)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)
        
        res += " " + tokenizer.index_word[indx]
        
    return res

In [14]:
res = build_phrase('лишь утратив всё')
print(res)

ValueError: cannot reshape array of size 1000 into shape (1,3,1000)